# Coffee Shop exploration (Within NH, VT, and ME)
### Nate Leavitt

## Table of contents
* [Introduction: Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Problem <a name="introduction"></a>
I would like to plan a trip that spans across VT, NH, and ME. During this trip, I want to experience the best coffee shops, based on popularity, each state has to offer, within 100,000 meters of each of the three state's capitals. I would also like to check the frequencies for the most common coffee shops, then cross compare the most popular to the most common/frequent. With both of these, I feel like I'll be able to map out the best route as I drive across each of the three states and find the most popular coffee shops that are NOT also the most common. (Please view the "Process" section below for more detail)

### Target Audience:
Coffee lovers who travel Northern New England.

### Problem to solve:
Avoid drinking a bad cup of coffee while on vacation.

---

## Data: <a name="data"></a>

The data needed to decided which coffee shop is worth going to while on our trip comes down to the popularity of a coffee shop and the number of same coffee shops within a given city.

After collecting the approprate data sets, we will need to map each coffee shop to best plan our route.

In order to source this data, we will use the FourSquare API, specifically the 'expole' end point. There will also be 5 parameters appened to this API call, including 'radius', 'limit', 'sortByPopularity', and 'categoryId' (these will be covered in more detail further in this section).

### Example data set:

Here is an example of what we will be looking for from our FourSquare API call.

In [ ]:
{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5e654b90ff57a30008ec77f1',
       'name': 'Starbucks', #Name, will be used, as mentioned above
       'location': {'address': '54 Sunderland Way',
        'lat': 44.504437,   #Lat, will be used, as mentioned above
        'lng': -73.138335,  #Lng, will be used, as mentioned above
        'labeledLatLngs': [{'label': 'display',
          'lat': 44.504437,
          'lng': -73.138335}],
        'distance': 6706,
        'postalCode': '05452',
        'cc': 'US',
        'city': 'Essex Junction', #City, will be used, as mentioned above
        'state': 'VT',      #Sate, will be used, as mentioned above
        'country': 'United States',
        'formattedAddress': ['54 Sunderland Way',
         'Essex Junction, VT 05452',
         'United States']},
       'categories': [{'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'shortName': 'Coffee Shop',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-5e654b90ff57a30008ec77f1-2'}

### Find each state's capital coordinates:

In [3]:
##Import all librairs needed:

import pandas as pd
import numpy as np

#Find latitudinal and longitudinal data for a given location
!pip install geopy
from geopy.geocoders import Nominatim

#--Addtional libraires for mapping:--
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json # library to handle JSON files

#--To communicate with FourSquare API--
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

# All requested packages already installed.



#### Vermont:

In [4]:
#Define location to look for
address = 'Burlington, Vermont'

geolocator = Nominatim(user_agent="vt_explorer")
location = geolocator.geocode(address)

#Save lat and lng in variables specific to Vermont
latitude_vt = location.latitude
longitude_vt = location.longitude
print('The geograpical coordinate of Vermont are {}, {}.'.format(latitude_vt, longitude_vt))

The geograpical coordinate of Vermont are 44.4761601, -73.212906.


#### New Hampshire:

In [5]:
#Define location to look for
address = 'Concord, New Hampshire'

geolocator = Nominatim(user_agent="nh_explorer")
location = geolocator.geocode(address)

#Save lat and lng in variables specific to New Hampshire
latitude_nh = location.latitude
longitude_nh = location.longitude
print('The geograpical coordinate of New Hampshire are {}, {}.'.format(latitude_nh, longitude_nh))

The geograpical coordinate of New Hampshire are 43.207178, -71.537476.


#### Maine:

In [6]:
#Define location to look for
address = 'Augusta, Maine'

geolocator = Nominatim(user_agent="me_explorer")
location = geolocator.geocode(address)

#Save lat and lng in variables specific to Maine
latitude_me = location.latitude
longitude_me = location.longitude
print('The geograpical coordinate of Maine are {}, {}.'.format(latitude_me, longitude_me))

The geograpical coordinate of Maine are 44.310545, -69.7792759.


Now let's use these coordinates to find some coffee shops, while using the FourSquare API.

First, we will need to setup our credentials for the API:

In [106]:
CLIENT_ID = '***' # your Foursquare ID
CLIENT_SECRET = '***' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ***
CLIENT_SECRET:***


Now we can define our endpoint parameters:
* LIMIT = 50 (The max is 50)
* radius = 5,000 meters (This is roughly the same as 31 miles)
* latitude_{state} = [44.4761601,43.207178,44.310545]
* longitude_{state} = [-73.212906,-71.537476,-69.7792759]
* sortByPopularity = 1 ("Boolean flag to sort the results by popularity instead of relevance.")
* categoryId = [4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735] (Specific categories of venues, to limit the expolre call)

### Find venues for each set of coordinates:

For each set of coordinates let's create the URL, submit a get requst, and store the desired columns into a dataframe.

Based on the example date shown above, under the 'Example data set' section, we can extract 'name', 'categories', 'city', 'state', 'lat' and 'lng'. In order to simplify the extraction of these values we will use a custom function, shown below:

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Vermont:

In [107]:
LIMIT = 50 # limit of number of venues returned by Foursquare API



radius = 50000 # define radius

# create URL
url_vt = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByPopularity=1&categoryId=4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_vt, 
    longitude_vt,
    radius,
    LIMIT)
url_vt # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=***&client_secret=***&v=20180605&ll=44.4761601,-73.212906&radius=50000&limit=50&sortByPopularity=1&categoryId=4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735'

In [9]:
results_vt = requests.get(url_vt).json()
results_vt

{'meta': {'code': 200, 'requestId': '60ef2bd6c4ec617459875acf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Burlington',
  'headerFullLocation': 'Burlington',
  'headerLocationGranularity': 'city',
  'query': 'coffee shop',
  'totalResults': 113,
  'suggestedBounds': {'ne': {'lat': 44.92616055000045,
    'lng': -72.58342626104405},
   'sw': {'lat': 44.02615964999955, 'lng': -73.84238573895595}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b2ec27498e609aa439d410',
       'name': 'UVM Medical Center Harvest Café',
       'location': {'address': '111 Colchester Ave',
        'crossStreet': 'Medical Center Campus, McClure Lobby',
     

In [49]:
venues_vt = results_vt['response']['groups'][0]['items']
    
nearby_venues_vt = json_normalize(venues_vt) # flatten JSON

<ipython-input-49-c3685445dbe3>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues_vt = json_normalize(venues_vt) # flatten JSON


In [50]:
# filter columns
filtered_columns_vt = ['venue.name', 'venue.categories', 'venue.location.city', 'venue.location.state', 'venue.location.lat', 'venue.location.lng']
nearby_venues_vt =nearby_venues_vt.loc[:, filtered_columns_vt]

# filter the category for each row
nearby_venues_vt['venue.categories'] = nearby_venues_vt.apply(get_category_type, axis=1)

# clean columns
nearby_venues_vt.columns = [col.split(".")[-1] for col in nearby_venues_vt.columns]

nearby_venues_vt.head()

,name,categories,city,state,lat,lng
0,UVM Medical Center Harvest Café,Café,Burlington,VT,44.480187,-73.195263
1,Starbucks,Coffee Shop,Essex Junction,VT,44.504437,-73.138335
2,Starbucks,Coffee Shop,South Burlington,VT,44.468840,-73.178050
3,Maplefields Mobile. US Ave,Coffee Shop,Plattsburgh,NY,44.682333,-73.447903
4,Vermont Artisan Coffee & Tea Co,Coffee Shop,Waterbury Center,VT,44.394481,-72.719934


#### New Hampshire:

In [109]:
LIMIT = 500 # limit of number of venues returned by Foursquare API



radius = 50000 # define radius

# create URL
url_nh = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByPopularity=1&categoryId=4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_nh, 
    longitude_nh,
    radius,
    LIMIT)
url_nh # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=***&client_secret=***&v=20180605&ll=43.207178,-71.537476&radius=50000&limit=500&sortByPopularity=1&categoryId=4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735'

In [13]:
results_nh = requests.get(url_nh).json()
results_nh

{'meta': {'code': 200, 'requestId': '60ef2c001bcbea3002e63154'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Concord',
  'headerFullLocation': 'Concord',
  'headerLocationGranularity': 'city',
  'query': 'coffee shop',
  'totalResults': 125,
  'suggestedBounds': {'ne': {'lat': 43.65717845000045,
    'lng': -70.92124495493779},
   'sw': {'lat': 42.75717754999955, 'lng': -72.1537070450622}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5dcc8744d4b62c00094208eb',
       'name': "Dunkin'",
       'location': {'address': '54 Park Ave',
        'lat': 43.22467752376902,
        'lng': -71.71416879017829,
        'labeledLatLngs': [{'label': 

In [21]:
venues_nh = results_nh['response']['groups'][0]['items']
    
nearby_venues_nh = json_normalize(venues_nh) # flatten JSON

<ipython-input-21-a5764f285550>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues_nh = json_normalize(venues_nh) # flatten JSON


In [22]:
# filter columns
filtered_columns_nh = ['venue.name', 'venue.categories', 'venue.location.city', 'venue.location.state', 'venue.location.lat', 'venue.location.lng']
nearby_venues_nh =nearby_venues_nh.loc[:, filtered_columns_nh]

# filter the category for each row
nearby_venues_nh['venue.categories'] = nearby_venues_nh.apply(get_category_type, axis=1)

# clean columns
nearby_venues_nh.columns = [col.split(".")[-1] for col in nearby_venues_nh.columns]

nearby_venues_nh.head()

,name,categories,city,state,lat,lng
0,Dunkin',Café,Hopkinton,NH,43.224678,-71.714169
1,Starbucks,Coffee Shop,Concord,NH,43.222623,-71.491814
2,Starbucks,Coffee Shop,Bedford,NH,42.956617,-71.478125
3,Heavenly Donuts,Coffee Shop,Derry,NH,42.879444,-71.300697
4,Pressed Cafe,Café,Nashua,NH,42.784077,-71.507550


#### Maine:

In [108]:
LIMIT = 50 # limit of number of venues returned by Foursquare API



radius = 50000 # define radius

# create URL
url_me = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByPopularity=1&categoryId=4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_me, 
    longitude_me,
    radius,
    LIMIT)
url_me # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=***&client_secret=***&v=20180605&ll=44.310545,-69.7792759&radius=50000&limit=50&sortByPopularity=1&categoryId=4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735'

In [18]:
results_me = requests.get(url_me).json()
results_me

{'meta': {'code': 200, 'requestId': '60ef35da43c9ba6c5bd3b3e7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Augusta',
  'headerFullLocation': 'Augusta',
  'headerLocationGranularity': 'city',
  'query': 'coffee shop',
  'totalResults': 59,
  'suggestedBounds': {'ne': {'lat': 44.76054545000045,
    'lng': -69.15157504644188},
   'sw': {'lat': 43.860544549999545, 'lng': -70.40697675355813}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '570ffd5fcd10b0249f84ced8',
       'name': 'Tim Hortons',
       'location': {'address': '165 Hinckley Rd,',
        'lat': 44.640163,
        'lng': -69.524944,
        'labeledLatLngs': [{'label': 'displ

In [23]:
venues_me = results_me['response']['groups'][0]['items']
    
nearby_venues_me = json_normalize(venues_me) # flatten JSON

<ipython-input-23-d176502235e9>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues_me = json_normalize(venues_me) # flatten JSON


In [24]:
# filter columns
filtered_columns_me = ['venue.name', 'venue.categories', 'venue.location.city', 'venue.location.state', 'venue.location.lat', 'venue.location.lng']
nearby_venues_me =nearby_venues_me.loc[:, filtered_columns_me]

# filter the category for each row
nearby_venues_me['venue.categories'] = nearby_venues_me.apply(get_category_type, axis=1)

# clean columns
nearby_venues_me.columns = [col.split(".")[-1] for col in nearby_venues_me.columns]

nearby_venues_me.head()

,name,categories,city,state,lat,lng
0,Tim Hortons,Coffee Shop,Clinton,ME,44.640163,-69.524944
1,The Olde Post Office Cafe,Café,Mount Vernom,ME,44.500338,-69.988051
2,Fiona's Catering & Take-Out,Café,Bath,ME,43.903829,-69.815884
3,Dunkin',Café,Winslow,ME,44.541528,-69.626393
4,Starbucks,Coffee Shop,Augusta,ME,44.314647,-69.804380


### Final cleaning:

Let's also do a final cleaning to remove any venues that might have been within our radius parameter, but not in our desired cities.

#### Vermont:

In [51]:
# get indexes for which column 'city' does not have Burlington
index_city = nearby_venues_vt[ nearby_venues_vt['city'] != 'Burlington' ].index
  
# drop these row indexes from dataFrame
nearby_venues_vt.drop(index_city, inplace = True)

nearby_venues_vt

,name,categories,city,state,lat,lng
0,UVM Medical Center Harvest Café,Café,Burlington,VT,44.480187,-73.195263
5,Kru Coffee,Café,Burlington,VT,44.480189,-73.212540
6,Brio Coffeeworks,Coffee Shop,Burlington,VT,44.472164,-73.214630
7,Starbucks,Coffee Shop,Burlington,VT,44.478731,-73.213944
13,Scout & Company,Coffee Shop,Burlington,VT,44.487890,-73.225259
14,Kestrel Coffee,Coffee Shop,Burlington,VT,44.473176,-73.217883
15,Nomad Coffee,Coffee Shop,Burlington,VT,44.456654,-73.217461
17,Speeder & Earl's,Coffee Shop,Burlington,VT,44.467849,-73.214927
18,UVM Hendersons Cafe,Café,Burlington,VT,44.475963,-73.196532
21,Onyx Tonics,Coffee Shop,Burlington,VT,44.477074,-73.214772


#### New Hampshire:

In [42]:
# get indexes for which column 'city' does not have Burlington
index_city = nearby_venues_nh[ nearby_venues_nh['city'] != 'Concord' ].index
  
# drop these row indexes from dataFrame
nearby_venues_nh.drop(index_city, inplace = True)

nearby_venues_nh

,name,categories,city,state,lat,lng
1,Starbucks,Coffee Shop,Concord,NH,43.222623,-71.491814
16,Aroma Joe's,Coffee Shop,Concord,NH,43.215183,-71.509802
18,The Works (The Works Café),Coffee Shop,Concord,NH,43.205418,-71.535897
48,White Mountain Coffee,Café,Concord,NH,43.203844,-71.536743
51,Revelstoke,Coffee Shop,Concord,NH,43.206997,-71.536150
59,In A Pinch,Café,Concord,NH,43.199543,-71.549771
67,Still In A Pinch,Café,Concord,NH,43.192401,-71.532597
74,True Brew Barista,Coffee Shop,Concord,NH,43.204784,-71.536799
75,Federal’s Cafe,Café,Concord,NH,43.202271,-71.540398
93,True Brew Cafe,Café,Concord,NH,43.202041,-71.534520


#### Maine

In [43]:
# get indexes for which column 'city' does not have Burlington
index_city = nearby_venues_me[ nearby_venues_me['city'] != 'Augusta' ].index
  
# drop these row indexes from dataFrame
nearby_venues_me.drop(index_city, inplace = True)

nearby_venues_me

,name,categories,city,state,lat,lng
4,Starbucks,Coffee Shop,Augusta,ME,44.314647,-69.804380
41,Vickery Cafe,Café,Augusta,ME,44.315570,-69.773739
43,Huiskamer Coffee House,Coffee Shop,Augusta,ME,44.316321,-69.773795
45,Main Street Café,Café,Augusta,ME,44.362900,-69.780930


That's the end of the data gathering phase. We are now ready to compare, analyze and map the venues within each city. 

---

## Methodology: <a name="methodology"></a>

In this project we will be looking for and mapping the most popular coffee shops within the capital cities of Vermont, New Hampshire, and Maine. We would also like to exclude from our 'popular list' any coffee shop that is a franchise (this list will include Starbucks, Aroma Joe's, White Mountain Coffee, and Dunkin'), but the franchise coffee shops will still be mapped using different color dots.

In the first step, we have gathered the geospacial coordinates for each of the three capitals, using Nominatim (from geopy.geocoders). These coordinates were then used within three seperate APT calls to FourSquare's 'explore' endpoint to gather the venue data. This call included the following parameters 'radius', 'limit', 'sortByPopularity', 'categoryId', and the lat and lng for each capital.

Secondly, the datasets were combed through to generate three seperate dataframes (one per capital), as well as a final cleaning that removed/dropped any venue that was not within the capital city (caused by the 'limit' parameter being too large).

Next, the franchised coffee shops were dropped from each of the capital dataframes. Once the franchised coffee shops were removed, the first row within each of the edited captial dataframes were copied to a new dataframe, designated only for the top three coffee shops.

Lastly, the top three dataframe, the franchised dataframe, as well as what was left within each of the capital dataframes were all mapped using folium, using the colors 'green', 'red', and 'blue', respectively.

## Analysis <a name="analysis"></a>

We want to find the most popular, but first we need to remove the franchised coffee shops from our main list. We don't want to delete these venues, just move them into a seperate list.

#### Vermont

In [52]:
#Create new dataframe for Vermont franchises
franchise_vt = nearby_venues_vt.loc[(nearby_venues_vt['name'] == 'Starbucks') | (nearby_venues_vt['name'] == "Aroma Joe's") | (nearby_venues_vt['name'] == "White Mountain Coffee") | (nearby_venues_vt['name'] == "Dunkin'")]

franchise_vt

,name,categories,city,state,lat,lng
7,Starbucks,Coffee Shop,Burlington,VT,44.478731,-73.213944
27,Starbucks,Coffee Shop,Burlington,VT,44.449005,-73.211006


In [53]:
#Drop franchise list from popular list
# get indexes for all franchised shops in Burlington
index_franchise = nearby_venues_vt[ (nearby_venues_vt['name'] == 'Starbucks') | (nearby_venues_vt['name'] == "Aroma Joe's") | (nearby_venues_vt['name'] == "White Mountain Coffee") | (nearby_venues_vt['name'] == "Dunkin'") ].index
  
# drop these row indexes from dataFrame
nearby_venues_vt.drop(index_franchise, inplace = True)

nearby_venues_vt

,name,categories,city,state,lat,lng
0,UVM Medical Center Harvest Café,Café,Burlington,VT,44.480187,-73.195263
5,Kru Coffee,Café,Burlington,VT,44.480189,-73.212540
6,Brio Coffeeworks,Coffee Shop,Burlington,VT,44.472164,-73.214630
13,Scout & Company,Coffee Shop,Burlington,VT,44.487890,-73.225259
14,Kestrel Coffee,Coffee Shop,Burlington,VT,44.473176,-73.217883
15,Nomad Coffee,Coffee Shop,Burlington,VT,44.456654,-73.217461
17,Speeder & Earl's,Coffee Shop,Burlington,VT,44.467849,-73.214927
18,UVM Hendersons Cafe,Café,Burlington,VT,44.475963,-73.196532
21,Onyx Tonics,Coffee Shop,Burlington,VT,44.477074,-73.214772
22,Dobra Tea,Tea Room,Burlington,VT,44.478213,-73.212494


#### New Hampshire

In [54]:
#Create new dataframe for Vermont franchises
franchise_nh = nearby_venues_nh.loc[(nearby_venues_nh['name'] == 'Starbucks') | (nearby_venues_nh['name'] == "Aroma Joe's") | (nearby_venues_nh['name'] == "White Mountain Coffee") | (nearby_venues_nh['name'] == "Dunkin'")]

franchise_nh

,name,categories,city,state,lat,lng
1,Starbucks,Coffee Shop,Concord,NH,43.222623,-71.491814
16,Aroma Joe's,Coffee Shop,Concord,NH,43.215183,-71.509802
48,White Mountain Coffee,Café,Concord,NH,43.203844,-71.536743


In [55]:
#Drop franchise list from popular list
# get indexes for all franchised shops in Concord
index_franchise = nearby_venues_nh[ (nearby_venues_nh['name'] == 'Starbucks') | (nearby_venues_nh['name'] == "Aroma Joe's") | (nearby_venues_nh['name'] == "White Mountain Coffee") | (nearby_venues_nh['name'] == "Dunkin'") ].index
  
# drop these row indexes from dataFrame
nearby_venues_nh.drop(index_franchise, inplace = True)

nearby_venues_nh

,name,categories,city,state,lat,lng
18,The Works (The Works Café),Coffee Shop,Concord,NH,43.205418,-71.535897
51,Revelstoke,Coffee Shop,Concord,NH,43.206997,-71.536150
59,In A Pinch,Café,Concord,NH,43.199543,-71.549771
67,Still In A Pinch,Café,Concord,NH,43.192401,-71.532597
74,True Brew Barista,Coffee Shop,Concord,NH,43.204784,-71.536799
75,Federal’s Cafe,Café,Concord,NH,43.202271,-71.540398
93,True Brew Cafe,Café,Concord,NH,43.202041,-71.534520


#### Maine

In [56]:
#Create new dataframe for Vermont franchises
franchise_me = nearby_venues_me.loc[(nearby_venues_me['name'] == 'Starbucks') | (nearby_venues_me['name'] == "Aroma Joe's") | (nearby_venues_me['name'] == "White Mountain Coffee") | (nearby_venues_me['name'] == "Dunkin'")]

franchise_me

,name,categories,city,state,lat,lng
4,Starbucks,Coffee Shop,Augusta,ME,44.314647,-69.80438


In [58]:
#Drop franchise list from popular list
# get indexes for all franchised shops in Augusta
index_franchise = nearby_venues_me[ (nearby_venues_me['name'] == 'Starbucks') | (nearby_venues_me['name'] == "Aroma Joe's") | (nearby_venues_me['name'] == "White Mountain Coffee") | (nearby_venues_me['name'] == "Dunkin'") ].index
  
# drop these row indexes from dataFrame
nearby_venues_me.drop(index_franchise, inplace = True)

nearby_venues_me

,name,categories,city,state,lat,lng
41,Vickery Cafe,Café,Augusta,ME,44.315570,-69.773739
43,Huiskamer Coffee House,Coffee Shop,Augusta,ME,44.316321,-69.773795
45,Main Street Café,Café,Augusta,ME,44.362900,-69.780930


Since we know which are the most popular, the first row in each nearby_venues_[state], becuase we used the "sortByPopularity" parameter during the "explore" FourSquare API, we can create another dataframe using a collection of the first rows.

In [89]:
#Use one dataframe to establish new dataframe
popular_venues = nearby_venues_vt.head(1)

#Append the other two
popular_venues = popular_venues.append([nearby_venues_nh.head(1),nearby_venues_me.head(1)], ignore_index=True)

popular_venues

,name,categories,city,state,lat,lng
0,UVM Medical Center Harvest Café,Café,Burlington,VT,44.480187,-73.195263
1,The Works (The Works Café),Coffee Shop,Concord,NH,43.205418,-71.535897
2,Vickery Cafe,Café,Augusta,ME,44.315570,-69.773739


### Mapping:

#### Define mapping scope:

We will first need to define where the map will be focused. To achieve this, let's use the coordinates from New Hampshire's capital, since it is located between the other two capitals. We can just pull these values from "latitude_nh" "longitude_nh", which we gathered back in subsection "Find each state's capital coordinates:", within the "Data" section.

In [82]:
# create map that encompasses all three states using the latitude and longitude values from 'Concord'
map_CoffeeShops = folium.Map(location=[latitude_nh, longitude_nh], zoom_start=7)

map_CoffeeShops

Let's first map our top three coffee shops, since these are our desired destinations. We will use "folium" to generate the map and plot each coffee shop, using the color green.

In [105]:
# add top three markers to map
for lat, lng, newClass, cat,  in zip(popular_venues['lat'], popular_venues['lng'], popular_venues['name'], popular_venues['categories']):
    label = folium.Popup('Name:' + newClass + ', category:' + cat, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CoffeeShops)  
    
    
map_CoffeeShops

Next, we'll map the franchise locations. We will color these red, to create a large contrast between our three desired locations. Before we do, we will need to combine our three franchise dataframes, from each capital, into one collection.

In [93]:
#Use one dataframe to establish new dataframe
franchise_venues = franchise_vt

#Append the other two
franchise_venues = franchise_venues.append([franchise_nh,franchise_me], ignore_index=True)

franchise_venues

,name,categories,city,state,lat,lng
0,Starbucks,Coffee Shop,Burlington,VT,44.478731,-73.213944
1,Starbucks,Coffee Shop,Burlington,VT,44.449005,-73.211006
2,Starbucks,Coffee Shop,Concord,NH,43.222623,-71.491814
3,Aroma Joe's,Coffee Shop,Concord,NH,43.215183,-71.509802
4,White Mountain Coffee,Café,Concord,NH,43.203844,-71.536743
5,Starbucks,Coffee Shop,Augusta,ME,44.314647,-69.804380


In [104]:
# add franchise markers to map
for lat, lng, newClass, cat,  in zip(franchise_venues['lat'], franchise_venues['lng'], franchise_venues['name'], franchise_venues['categories']):
    label = folium.Popup('Name:' + newClass + ', category:' + cat, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CoffeeShops)  

    
map_CoffeeShops

Just for fun, we can also map the rest of the venues pulled. Let's add them using blue dots. Before we do, we will need to combine all of the rest of the venues.

In [95]:
#Use one dataframe to establish new dataframe
other_venues = nearby_venues_vt

#Append the other two
other_venues = other_venues.append([nearby_venues_nh,nearby_venues_me], ignore_index=True)

other_venues

,name,categories,city,state,lat,lng
0,UVM Medical Center Harvest Café,Café,Burlington,VT,44.480187,-73.195263
1,Kru Coffee,Café,Burlington,VT,44.480189,-73.212540
2,Brio Coffeeworks,Coffee Shop,Burlington,VT,44.472164,-73.214630
3,Scout & Company,Coffee Shop,Burlington,VT,44.487890,-73.225259
4,Kestrel Coffee,Coffee Shop,Burlington,VT,44.473176,-73.217883
5,Nomad Coffee,Coffee Shop,Burlington,VT,44.456654,-73.217461
6,Speeder & Earl's,Coffee Shop,Burlington,VT,44.467849,-73.214927
7,UVM Hendersons Cafe,Café,Burlington,VT,44.475963,-73.196532
8,Onyx Tonics,Coffee Shop,Burlington,VT,44.477074,-73.214772
9,Dobra Tea,Tea Room,Burlington,VT,44.478213,-73.212494


In [99]:
#Drop top coffee shops from 'other' list
# get indexes for the top three coffee shops
index_topThree = other_venues[ (other_venues['name'] == 'UVM Medical Center Harvest Café') | (other_venues['name'] == "The Works (The Works Café)") | (other_venues['name'] == "Vickery Cafe")].index
  
# drop these row indexes from dataFrame
other_venues.drop(index_topThree, inplace = True)

other_venues

,name,categories,city,state,lat,lng
1,Kru Coffee,Café,Burlington,VT,44.480189,-73.212540
2,Brio Coffeeworks,Coffee Shop,Burlington,VT,44.472164,-73.214630
3,Scout & Company,Coffee Shop,Burlington,VT,44.487890,-73.225259
4,Kestrel Coffee,Coffee Shop,Burlington,VT,44.473176,-73.217883
5,Nomad Coffee,Coffee Shop,Burlington,VT,44.456654,-73.217461
6,Speeder & Earl's,Coffee Shop,Burlington,VT,44.467849,-73.214927
7,UVM Hendersons Cafe,Café,Burlington,VT,44.475963,-73.196532
8,Onyx Tonics,Coffee Shop,Burlington,VT,44.477074,-73.214772
9,Dobra Tea,Tea Room,Burlington,VT,44.478213,-73.212494
10,Vivid Coffee,Coffee Shop,Burlington,VT,44.479484,-73.212106


In [100]:
#Reindex to remove gaps from dropping the top three
other_venues.reset_index(drop=True)

,name,categories,city,state,lat,lng
0,Kru Coffee,Café,Burlington,VT,44.480189,-73.212540
1,Brio Coffeeworks,Coffee Shop,Burlington,VT,44.472164,-73.214630
2,Scout & Company,Coffee Shop,Burlington,VT,44.487890,-73.225259
3,Kestrel Coffee,Coffee Shop,Burlington,VT,44.473176,-73.217883
4,Nomad Coffee,Coffee Shop,Burlington,VT,44.456654,-73.217461
5,Speeder & Earl's,Coffee Shop,Burlington,VT,44.467849,-73.214927
6,UVM Hendersons Cafe,Café,Burlington,VT,44.475963,-73.196532
7,Onyx Tonics,Coffee Shop,Burlington,VT,44.477074,-73.214772
8,Dobra Tea,Tea Room,Burlington,VT,44.478213,-73.212494
9,Vivid Coffee,Coffee Shop,Burlington,VT,44.479484,-73.212106


In [103]:
# add franchise markers to map
for lat, lng, newClass, cat,  in zip(other_venues['lat'], other_venues['lng'], other_venues['name'], other_venues['categories']):
    label = folium.Popup('Name:' + newClass + ', category:' + cat, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CoffeeShops)  

    
map_CoffeeShops

## Results and Discussion <a name="results"></a>

These results are pretty straight forward. We found the most popular coffee shops within each capital city:

* For Burlinton, Vermont we are suggesting "UVM Medical Center Harvest Café"
* For Concord, New Hampshire we are suggesting "The Works (The Works Café)"
* For Augusta, Maine we are suggesting "Vickery Cafe"

Using the "sortByPopularity" parameter to sort the pulled venues by popularity gave us good foundation to work with. From there, most of the work was around identifying the franchised coffee shops and droping them from our main lists. Once the franchised coffee shops were removed, we could then pull the first position row from each of the three capital's dataframes, and save them to a new dataframe for just the top three. This then allowed us to easily map/visualize exactly where each of the top three coffee shops were located.

## Conclusion <a name="conclusion"></a>

The purpose of this project was to identiy the most popular non-franchised coffee shops within three different capital cities, as well as visualize their locations, so that individuals who enjoy coffee and who are traveling between Burlinton Vermont, Concord New Hampshire, and Augusta Maine could easily identify a potentially good place to get a cup of coffee, while avoinding any franchise.

Somethings to consider for future studies would be adding suggested routes between the top three shops. Maybe using something like the networkx.shortest_path() method to generate a route using existing roads would be helpful to travelers (https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.plot.plot_route_folium). 

Another topic that might be worth exploring is to dig deeper into the meta data around each of the top three coffee shops. This could provide some guidance around why these three are the top three. Maybe it's based on their location to a college campus, or a hospital. Either way, using the "details" FourSquare end point of each of the top three venues might be the next best step (https://developer.foursquare.com/docs/api-reference/venues/details/).